<a href="https://colab.research.google.com/github/evinracher/3010090-ontological-engineering/blob/main/week3/part1/3_01_LangGraph_Checkpointing_TimeTravel_Streaming_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph: Checkpointing, Thread IDs, Time Travel y Streaming (Gemini)
Este cuaderno complementa los ejemplos anteriores y cubre **las secciones desde checkpointing en adelante**:
- Checkpointing (InMemory/SQLite)
- Thread IDs y multi-usuario
- Time travel: ver historial, volver a un checkpoint y editar estado
- Streaming en LangGraph: `values`, `updates`, `messages`

In [1]:
%pip install -U langgraph langchain-google-genai pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.4
    Uninstalling pydantic_core-2.41.4:
      Successfully uninstalled pydantic_core-2.41.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.3
    Uninstalling pydantic-2.12.3:
      Successfully uninstalled pydantic-2.12.3
  Attempting uninstall: langgraph-prebuilt
    Found existing installation: langgraph-prebuilt 1.0.7
    Uninstalling langgraph-prebuilt-1.0.7:
      Successfully uninstalled langgraph-prebuilt-1.0.7
  Attempting uninstall: langgraph
    Found existing inst

In [2]:
from google.colab import userdata
import os

api_key = userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = api_key
print('API Key cargada:', 'Sí' if api_key else 'No')

API Key cargada: Sí


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Configure the Gemini API
MODEL_ID = os.getenv("GEMINI_MODEL", "models/gemini-2.5-flash-lite")

llm = ChatGoogleGenerativeAI(model=MODEL_ID, temperature=0.2)
print("✅ LLM listo:", MODEL_ID)

✅ LLM listo: models/gemini-2.5-flash-lite


## 1) Checkpointing: guardar el estado automáticamente
Un **checkpointer** guarda snapshots del estado del grafo por `thread_id`.
- En dev: `InMemorySaver`
- Persistencia local: `SqliteSaver`

In [4]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    topic: str
    joke: str

def make_topic(state: State):
    return {"topic": state["topic"].strip()}

def make_joke(state: State):
    # Nodo que usa el LLM
    prompt = f"Cuenta un chiste corto sobre: {state['topic']}. Solo una frase."
    resp = llm.invoke(prompt)
    return {"joke": resp.content}

builder = StateGraph(State)
builder.add_node("make_topic", make_topic)
builder.add_node("make_joke", make_joke)
builder.add_edge(START, "make_topic")
builder.add_edge("make_topic", "make_joke")
builder.add_edge("make_joke", END)

In [5]:
# Checkpointer en memoria
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "demo-1"}}

out = graph.invoke({"topic": "gatos"}, config=config)
out

{'topic': 'gatos',
 'joke': '¿Por qué los gatos son malos jugadores de póker? Porque siempre tienen un "as" en la manga... ¡y lo usan para rascar!'}

### Ver el estado guardado (snapshot)

In [6]:
snapshot = graph.get_state(config)
snapshot.values

{'topic': 'gatos',
 'joke': '¿Por qué los gatos son malos jugadores de póker? Porque siempre tienen un "as" en la manga... ¡y lo usan para rascar!'}

### Ver historial de checkpoints (state history)

In [7]:
history = list(graph.get_state_history(config))
len(history), history[0].values

(4,
 {'topic': 'gatos',
  'joke': '¿Por qué los gatos son malos jugadores de póker? Porque siempre tienen un "as" en la manga... ¡y lo usan para rascar!'})

## 2) Thread IDs y multi-usuario
Cada usuario/sesión debe tener su propio `thread_id` para evitar mezclar estados.

In [8]:
config_carlos = {"configurable": {"thread_id": "user-carlos"}}
config_maria  = {"configurable": {"thread_id": "user-maria"}}

_ = graph.invoke({"topic": "fútbol"}, config=config_carlos)
_ = graph.invoke({"topic": "pizza"},  config=config_maria)

print("Carlos:", graph.get_state(config_carlos).values)
print("María :", graph.get_state(config_maria).values)

Carlos: {'topic': 'fútbol', 'joke': '¿Por qué el balón de fútbol fue al psicólogo? Porque tenía muchos problemas de "golpeo".'}
María : {'topic': 'pizza', 'joke': '¿Por qué la pizza fue a terapia? Porque tenía muchos problemas de *masa*.'}


## 3) Time travel
Con checkpointing activo puedes:
- Pedir el historial
- Volver a un `checkpoint_id`
- Editar el estado con `update_state` (fork)

In [9]:
# Tomemos el checkpoint más reciente y uno anterior (si existe)
history = list(graph.get_state_history(config))
latest = history[0]
older  = history[-1]

latest_id = latest.config["configurable"]["checkpoint_id"]
older_id  = older.config["configurable"]["checkpoint_id"]

print("latest_id:", latest_id)
print("older_id :", older_id)

latest_id: 1f10f20f-2061-6ab7-8002-7515fb9c2645
older_id : 1f10f20f-19fe-6fc8-bfff-09ad100218dd


### 3.1 Reproducir desde un checkpoint
Si invocas con `checkpoint_id`, LangGraph re-playea lo anterior y continúa desde ahí.

LangGraph va generando los ids de los nodos que se han ejecutado

In [ ]:
replay_config = {"configurable": {"thread_id": "demo-1", "checkpoint_id": older_id}}

# Nota: inputs puede ser None si el grafo no requiere nuevas entradas; aquí lo dejamos igual por claridad.
replayed = graph.invoke({"topic": "gatos"}, config=replay_config)
replayed

### 3.2 Editar estado (fork) con update_state
Ejemplo: cambiamos el topic manualmente y generamos otro chiste.

In [10]:
# Actualiza el estado "como si" viniera del nodo make_topic
graph.update_state(
    {"configurable": {"thread_id": "demo-1"}},
    {"topic": "perros"},
    as_node="make_topic"
)

# Ahora corremos desde el estado actual: debería regenerar joke
out2 = graph.invoke({"topic": "perros"}, config={"configurable": {"thread_id": "demo-1"}})
out2

{'topic': 'perros',
 'joke': '¿Por qué los perros no pueden bailar? Porque no tienen ritmo, ¡solo ladran!'}

## 4) Persistencia con SQLite (SqliteSaver)
Útil cuando quieres reiniciar el runtime y mantener estado.

In [11]:
%pip -q install -U langgraph langgraph-checkpoint-sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 12.6 MB/s eta 0:00:00


In [12]:
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "/content/langgraph_checkpoints.sqlite"

with SqliteSaver.from_conn_string(db_path) as sqlite_cp:
    graph_sqlite = builder.compile(checkpointer=sqlite_cp)

    cfg = {"configurable": {"thread_id": "sqlite-demo"}}
    out = graph_sqlite.invoke({"topic": "programación"}, config=cfg)
    print(out)
    print(graph_sqlite.get_state(cfg).values)

{'topic': 'programación', 'joke': '¿Por qué los programadores prefieren el modo oscuro? Porque la luz atrae a los bugs.'}
{'topic': 'programación', 'joke': '¿Por qué los programadores prefieren el modo oscuro? Porque la luz atrae a los bugs.'}


## 5) Streaming en LangGraph
Modos clave:
- `updates`: solo cambios por nodo
- `values`: estado completo por paso
- `messages`: tokens del LLM (typing effect)

In [13]:
# 5.1 updates
for chunk in graph.stream({"topic": "café"}, config={"configurable": {"thread_id": "stream-1"}}, stream_mode="updates"):
    print(chunk)

{'make_topic': {'topic': 'café'}}
{'make_joke': {'joke': '¿Por qué el café fue a terapia? Porque tenía demasiados problemas de "grano".'}}


In [14]:
# 5.2 values
for chunk in graph.stream({"topic": "café"}, config={"configurable": {"thread_id": "stream-2"}}, stream_mode="values"):
    print(chunk)

{'topic': 'café'}
{'topic': 'café'}
{'topic': 'café', 'joke': '¿Por qué el café fue a terapia? Porque tenía demasiados problemas de "grano".'}


In [15]:
# 5.3 messages (token streaming)
for msg_chunk, meta in graph.stream(
    {"topic": "café"},
    config={"configurable": {"thread_id": "stream-3"}},
    stream_mode="messages",
):
    # msg_chunk suele ser un AIMessageChunk con .content parcial
    if hasattr(msg_chunk, "content") and msg_chunk.content:
        print(msg_chunk.content, end="")

¿Por qué el café fue a terapia? Porque tenía demasiados problemas de "grano".